In [6]:
# imports
import os, sys, pathlib
root = pathlib.Path.cwd()
if (root / "PLD_subsampling").exists():
    sys.path.insert(0, str(root))
elif (root.parent / "PLD_subsampling").exists():
    sys.path.insert(0, str(root.parent))

import numpy as np
from PLD_subsampling.testing.test_utils import run_multiple_experiments, run_experiment
from PLD_subsampling.testing.plot_utils import create_pmf_cdf_plot, create_epsilon_delta_plot, print_experiment_table
from PLD_subsampling.testing.analytic_Gaussian import Gaussian_epsilon_for_delta

ModuleNotFoundError: No module named 'PLD_subsampling.testing'; 'PLD_subsampling' is not a package

# Run a single experiment ($\sigma=0.5$, $\lambda=0.1$, remove direction)

In [ ]:
sigma = 0.5
q = 0.1
discretization = 1e-4
deltas = np.array([10 ** (-k) for k in range(2, 13)], dtype=float)
remove_direction = True
versions = run_experiment(sigma=sigma, sampling_prob=q, discretization=discretization, delta_values=deltas, remove_direction=remove_direction)

NameError: name 'np' is not defined

In [ ]:
print(f"\nσ={sigma}, q={q}, disc={discretization:g}, dir={'rem' if remove_direction else 'add'}")
eps_GT = [
    Gaussian_epsilon_for_delta(sigma=sigma, sampling_prob=q, delta=float(d), remove_direction=remove_direction)
    for d in deltas
]
print_experiment_table(deltas, versions, eps_GT)

fig_cdf = create_pmf_cdf_plot(versions=versions, title_suffix=f'sigma={sigma}, q={q}, disc={discretization:.0e}, dir={'rem' if remove_direction else 'add'}')
fig_cdf.show()

In [ ]:
fig_eps = create_epsilon_delta_plot(delta_values=deltas, versions=versions, eps_GT=eps_GT, log_x_axis=True, log_y_axis=False, title_suffix=f'sigma={sigma}, q={q}, disc={discretization:.0e}, dir:{'rem' if remove_direction else 'add'}')
fig_eps.show()

# Run many experiments  ($\sigma=2.0$, $\lambda=0.5$)

In [ ]:
discretizations = [1e-4]
q_values = [0.5]
sigma_values = [2.0]
remove_directions = [True, False]
delta_values_arr = np.array([10 ** (-k) for k in range(2, 13)], dtype=float)
results = run_multiple_experiments(discretizations, q_values, sigma_values, remove_directions, delta_values_arr)

### Results for remove direction

In [ ]:
res = results[0]
sigma = res['sigma']; q = res['q']; discretization = res['discretization']; remove_direction = res['remove_direction']
versions = res['versions']; deltas = res['delta_values']
print(f"\nσ={sigma}, q={q}, disc={discretization:g}, dir={'rem' if remove_direction else 'add'}")
eps_GT = [
    Gaussian_epsilon_for_delta(sigma=sigma, sampling_prob=q, delta=float(d), remove_direction=remove_direction)
    for d in deltas
]
print_experiment_table(deltas, versions, eps_GT)

fig_cdf = create_pmf_cdf_plot(versions=versions, title_suffix=f'sigma={sigma}, q={q}, disc={discretization:.0e}, dir={'rem' if remove_direction else 'add'}')
fig_cdf.show()

### Results for add direction

In [ ]:
res = results[1]
sigma = res['sigma']; q = res['q']; discretization = res['discretization']; remove_direction = res['remove_direction']
versions = res['versions']; deltas = res['delta_values']
print(f"\nσ={sigma}, q={q}, disc={discretization:g}, dir={'rem' if remove_direction else 'add'}")
eps_GT = [
    Gaussian_epsilon_for_delta(sigma=sigma, sampling_prob=q, delta=float(d), remove_direction=remove_direction)
    for d in deltas
]
print_experiment_table(deltas, versions, eps_GT)

fig_cdf = create_pmf_cdf_plot(versions=versions, title_suffix=f'sigma={sigma}, q={q}, disc={discretization:.0e}, dir={'rem' if remove_direction else 'add'}')
fig_cdf.show()

# Transform a dp_accounting PLD object

In [ ]:
# PLD to PLD transformation (wrappers)
from PLD_subsampling.wrappers.dp_accounting_wrappers import (
    create_pld_and_extract_pmf,
    dp_accounting_pmf_to_loss_probs,
    loss_probs_to_dp_accounting_pmf,
    amplify_pld_separate_directions,
)

sigma = 1.0
q = 0.2
discretization = 1e-4
# Base non-amplified PLD
base_pmf = create_pld_and_extract_pmf(standard_deviation=sigma, sensitivity=1.0, sampling_prob=1.0, value_discretization_interval=discretization, remove_direction=True)
# Amplify separately for remove/add directions
amplified = amplify_pld_separate_directions(base_pld=base_pmf._parent, sampling_prob=q)
remove_amplified = amplified['pmf_remove']
add_amplified = amplified['pmf_add']

# Convert to losses/probs for inspection/plotting
losses_rem, probs_rem = dp_accounting_pmf_to_loss_probs(remove_amplified)
losses_add, probs_add = dp_accounting_pmf_to_loss_probs(add_amplified)
print(f"Remove direction: grid size={len(losses_rem)}, mass={probs_rem.sum():.6f}")
print(f"Add direction   : grid size={len(losses_add)}, mass={probs_add.sum():.6f}")
